In [31]:
from playwright.async_api import async_playwright
from urllib import parse

# Update python path to include the parent directory
import os
import sys
import datetime
from datetime import timedelta

sys.path.append("..")

os.environ["ROOT_PATH"] = "/Users/aennassiri/Projects/tickettakedown/app"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/aennassiri/Projects/tickettakedown/app/configuration/fubloo-app-1f213ca274de.json"
)

In [32]:
import requests

import pandas as pd
from src.services.leads import get_leads, get_last_lead, patch_lead

/var/folders/kb/yw9y12616lz4f5jp49qvpmmm0000gn/T/ipykernel_46989/1723853763.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Define Variables


In [116]:
LOGIN_PAGE_URL = "https://www.beenverified.com/app/login"
HOME_PAGE_URL = "https://www.beenverified.com/rf/dashboard"
REPORT_PAGE_URL = "https://www.beenverified.com/api/v5/reports"
SEARCH_PAGE_URL = "https://www.beenverified.com/rf/search/person"

In [117]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)



In [64]:
context = await browser.new_context(storage_state="playwright/.auth/state.json")
page =  await context.new_page()
await page.goto(HOME_PAGE_URL)

<Response url='https://www.beenverified.com/rf/dashboard' request=<Request url='https://www.beenverified.com/rf/dashboard' method='GET'>>

## Search for lead 

In [26]:
def build_search_url(
    first_name,
    last_name,
    middle_name: str | None,
    city: str | None = None,
    state: str | None = None,
    age: int | None = None,
):
    params = {}
    if city:
        params['city'] = city
    if state:
        params['state'] = state
    if age:
        params['age'] = age
    if first_name:
        params['fname'] = first_name
    if last_name:
        params['ln'] = last_name
    if middle_name:
        params['mn'] = middle_name
    
    url = parse.urlencode(params)
    url = f"{SEARCH_PAGE_URL}?{url}"

    return url

In [27]:
search_query = {
    "first_name": "VINCENT",
    "last_name": "WATKINS",
    "city": "",
    "state": "",
    "age": None
}

url = build_search_url(search_query["first_name"], search_query["last_name"], search_query["city"], search_query["state"], search_query["age"])
await page.goto(url)

In [49]:
search_query = {
    "first_name": "VINCENT",
    "last_name": "WATKINS",
    "city": "",
    "state": "",
    "age": None,
}

# get top 5 users

In [29]:
def score_of_leads(result, search_query):
    score = 0

    if search_query.get("first_name").lower() not in result.lower() or search_query.get("last_name").lower() not in result.lower() or "deceased" in result.lower():
        score = -1
        return score

    score += result.lower().count("alias")
    score += result.lower().count("relatives")
    score += result.lower().count(search_query.get("city").lower() if search_query.get("city") else "")
    score += result.lower().count(search_query.get("state").lower() if search_query.get("state") else "")
    score += result.lower().count(search_query.get("age").lower() if search_query.get("age") else "")
    score += result.lower().count(search_query.get("middle_name").lower() if search_query.get("middle_name") else "")

    return score


In [45]:
container_results = await page.query_selector(".css-ts1zsd")
container_results

<JSHandle preview=JSHandle@node>

In [46]:
container_users = await container_results.query_selector_all(".css-1mvdt3q")
#len of leads matched
len(container_users)

2

## filter by score lead


In [47]:
txt = await container_users[0].text_content()
txt

'Name matchVincent Mrk Watkins, 60Lake Ozark, MOBornSep 1963LocationsLake Ozark, MO • Camdenton, MO • Kaiser, MO • +4 moreView Person Report'

In [48]:
#score of leads
score_of_leads(txt, search_query)


-1

In [139]:
for lead in container_users:
    print(await lead.text_content())
    print("----")

Name matchSamama B Mahmud, 28Morton Grove, ILBornJul 1995RelativesAbdullah Al Mahmud, Labiba B Mahmud, Sharif Mahmud, +2 MoreLocationsMorton Grove, IL • Milwaukee, WIView Person Report
----


## Get info leasd

In [70]:
async def open_report_popup(page, container_user):
    async with page.expect_popup() as popup_info:
        view_report = await container_user.query_selector(".css-gnnc3j")
        await view_report.click()
        new_page = await popup_info.value
        return new_page


In [71]:
async def open_report_page(page, container_user):
    async with page.expect_popup() as popup_info:
        view_report = await container_user.query_selector(".css-gnnc3j")
        await view_report.click()
        new_page = await popup_info.value
        return new_page



# Exetra information of the user

In [74]:
async def extract_phone_numbers(page):
    container_phone = await page.query_selector("#phone-numbers-section")
    phone_numbers = await container_phone.query_selector_all(".css-1vugsqn")
    phone_numbers = [await phone_number.text_content() for phone_number in phone_numbers]
    return phone_numbers




In [75]:
async def extract_addresses(page):
    addresses_list = []
    container_address = await page.query_selector("#address-history-section")
    addresses = await container_address.query_selector_all(".css-1q4wjho")
    for address in addresses:
        address_fields = await address.query_selector_all(".css-zv7ju9")
        addresses_txt = [await address_field.text_content() for address_field in address_fields]
        addresses_list.append(' '.join(addresses_txt))
        
    return addresses_list



In [105]:
async def extract_email_list(page):
   
    email_container = await page.query_selector("#email-section")
    email_elements = await email_container.query_selector_all(".css-1vugsqn")
    email_list = [await email_element.text_content() for email_element in email_elements]
    return email_list



In [109]:
async def get_lead_info(new_page):
    extra_phone_numbers = await extract_phone_numbers(new_page)  # noqa: F821
    extra_addresses = await extract_addresses(new_page)
    extra_emails = await extract_email_list(new_page)
    return extra_phone_numbers, extra_addresses, extra_emails

# extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
# extra_phone_numbers, extra_addresses, extra_emails

    
    
    

## Get info of the user

In [110]:
for lead in container_users:
    print(await lead.text_content())
    new_page = await open_report_page(page, lead)
    await new_page.wait_for_selector("#person-overview")
    extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
    print(extra_phone_numbers, extra_addresses, extra_emails)
    print("----")
    # await new_page.close()

Name matchSamama B Mahmud, 28Morton Grove, ILBornJul 1995RelativesAbdullah Al Mahmud, Labiba B Mahmud, Sharif Mahmud, +2 MoreLocationsMorton Grove, IL • Milwaukee, WIView Person Report
JSHandle@<button tabindex="0" type="button" class="MuiButtonBase-…>…</button>
['(847) 521-9814', '(847) 581-0171'] ['5808 Keeney St Morton Grove, IL 60053', '811 22nd St Apt 11 Milwaukee, WI 53233'] ['samamamahmud17@gmail.com', 'samamamahmud@yahoo.com', 'samama.mahmud@marquette.edu']
----


In [102]:
extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
print(extra_phone_numbers, extra_addresses, extra_emails)

['(847) 521-9814', '(847) 581-0171'] ['5808 Keeney St Morton Grove, IL 60053', '811 22nd St Apt 11 Milwaukee, WI 53233'] ['samamamahmud17@gmail.com', 'samamamahmud@yahoo.com', 'samama.mahmud@marquette.edu']


In [112]:
await new_page.close()

In [30]:
# current from (314) 691-4319 to +13146914319
def format_phone_number(phone_number):
    output = (
        phone_number.replace("(", "")
        .replace(")", "")
        .replace(" ", "")
        .replace("-", "")
    )

    if len(output) == 10:
        return f"+1{output}"
    else:
        return output

In [69]:
consecutive_timeouts = 0

async def search_person(
    first_name,
    last_name,
    middle_name,
    age,
    city="",
    state="",
    zip="",
    addressLine1="",
    addressLine2="",
):
    global consecutive_timeouts
    search_query = {
        "first_name": str(first_name),
        "last_name": str(last_name),
        "middle_name": str(middle_name),
        "age": str(age),
        "city": str(city),
        "state": str(state),
        "zip": str(zip),
        "addressLine1": str(addressLine1),
        "addressLine2": str(addressLine2),
        
    }
    # Triggering the search
    url = build_search_url(
        first_name, last_name, middle_name, city, state, age
    )
    await page.goto(url)

    # Getting results
    try:
        await page.wait_for_selector(".css-ts1zsd")
    except Exception as e:
        print("Timeout")
        consecutive_timeouts += 1
        if consecutive_timeouts > 20:
            print("Too many timeouts")
            return
        print("No leads found")
        return
    container_results = await page.query_selector(".css-ts1zsd")
    
    if container_results is None: 
        print("No leads found")
        return
    
    try:
        await container_results.wait_for_selector(".css-1mvdt3q")
    except Exception as e:
        print("Timeout")
        consecutive_timeouts += 1
        if consecutive_timeouts > 20:
            print("Too many timeouts")
            return
        print("No leads found")
        return
    
    container_users = await container_results.query_selector_all(
        ".css-1mvdt3q"
    )

    consecutive_timeouts = 0

    # Measure the score for each lead
    max_score = -1
    max_score_id = -1
    for i, lead in enumerate(container_users):
        results = await lead.text_content()
        score = score_of_leads(results, search_query)
        print(f"Score of lead {i} is {score}")
        if score > max_score:
            max_score = score
            max_score_id = i

    print(f"Max score is {max_score} for lead {max_score_id}")

    if max_score < 0:
        print("No leads found")
        return

    # Extracting the leads
    selected_lead = container_users[max_score_id]
    text = await selected_lead.text_content()
    print(text)

    # Opening the report page
    new_page = await open_report_page(page, selected_lead)
    await new_page.wait_for_selector("#person-overview")

    # Extracting the extra info
    extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
    print(extra_phone_numbers, extra_addresses, extra_emails)

    results = {
        "phone_numbers": extra_phone_numbers,
        "addresses": extra_addresses,
        "emails": extra_emails
    }

    for p in context.pages[1:]:
        await p.close()

    return results

In [62]:
today = datetime.datetime.now()

In [71]:
while True: 
    lead = get_last_lead(
        start_date=today - timedelta(days=7),
        end_date=today + timedelta(days=1),
        status="new",
        limit=1,
    )

    lead.state = lead.state or "MO"

    results = await search_person(
        first_name=lead.first_name,
        last_name=lead.last_name,
        middle_name=lead.middle_name,
        age=lead.age,
        city=lead.city,
        state=lead.state,
    )

    if results:
        details = {
            "phones": [format_phone_number(p) for p in results["phone_numbers"]],
            "phone": {
                str(k): {
                    "phone": format_phone_number(p),
                }
                for k, p in enumerate(results["phone_numbers"])
            },
            "emails": results["emails"],
            "report": {
                "addresses": results["addresses"],
            },
            "lead_source": "beenverified",
            "status": "not_contacted",
        }
        patch_lead(case_id=lead.case_id, **details)

    else:
        patch_lead(case_id=lead.case_id, status="not_found", lead_source="beenverified")


Score of lead 0 is 22
Max score is 22 for lead 0
Name matchHollie R Cooper, 33Sullivan, MOBornOct 1990RelativesGary Wade Cooper, Jacob S Cooper, Jesse L Cooper, +2 MoreLocationsSullivan, MO • Troy, MO • Foley, MO • +2 moreView Person Report
['(636) 234-5898', '(281) 703-1959', '(440) 862-4684', '(573) 860-0485', '(979) 571-3242', '(636) 388-5537', '(573) 468-6366', '(817) 624-9955', '(636) 290-8987'] ['86 Skyline Dr Saint Charles, MO 63304', '136 Colonial Ln Apt 4 Troy, MO 63379', '1269 Dodgeville Rd # Fw Rome, OH 44085', '401 Maple St Apt 3 Sullivan, MO 63080', '1660 Elmont Rd Lot 2 Sullivan, MO 63080', '72 Ira Bland Apt 12 Sullivan, MO 63080', '100 Central Dr Foley, MO 63347', '1660 Elmont Rd Sullivan, MO 63080', '10750 Moss Oak Rd Sullivan, MO 63080', '1240 Marchi Dr Saint Louis, MO 63125'] ['cooperhollie282@gmail.com', 'holliecoop30@gmail.com', 'pguiffrida2010@gmail.com', 'naughty_lilgirl_2003@gmail.com', 'naughtylilgirl2003@yahoo.com', 'holliecooper1990@gmail.com', 'jaysmommi2008@

Error: Page crashed

In [56]:
lead.case_id

'SOLIS-MARTINEZ__DAVID_39_02_18_2024'

In [57]:
details

{'phones': ['+12096060589', '+12098921814', '+15104819427', '+13034130557'],
 'phone': {'0': {'phone': '+12096060589'},
  '1': {'phone': '+12098921814'},
  '2': {'phone': '+15104819427'},
  '3': {'phone': '+13034130557'}},
 'emails': ['karlasol1979@yahoo.com',
  'larthatta@gmail.com',
  'gulchera124000@yahoo.com'],
 'report': {'addresses': ['1346 Pinto Way Patterson, CA 95363',
   '1201 Yellowhammer Dr Patterson, CA 95363',
   '3563 Franklin Ave Fremont, CA 94538',
   '1674 E St Hayward, CA 94541',
   '132 Paramatta Dr Patterson, CA 95363',
   '15610 Via Granada San Lorenzo, CA 94580',
   '606 Trout Creek Ln Patterson, CA 95363',
   '15861 Hesperian Blvd San Lorenzo, CA 94580']},
 'lead_source': 'beenverified',
 'status': 'not_contacted'}

In [34]:
lead = get_last_lead(
    start_date=today - timedelta(days=7),
    end_date=today + timedelta(days=1),
    status="new",
    limit=1,
)

/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:145: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", ">=", start_date)
/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:149: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", "<=", end_date)
/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:151: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("status", "==", status)


In [113]:
await browser.close()
await pw.stop()

In [35]:
lead

Lead(id=None, case_id='WATKINS__VINCENT_M_64_02_18_2024', first_name='VINCENT', last_name='WATKINS', middle_name='M', court_code='temp', age=64, year_of_birth=1960, email=None, phone=None, phones=None, address=None, city=None, state=None, zip_code=None, county=None, creation_date=datetime.datetime(2024, 2, 18, 23, 17, 56, 652086), last_updated=DatetimeWithNanoseconds(2024, 2, 18, 10, 1, 10, 749549, tzinfo=datetime.timezone.utc), case_date=datetime.datetime(2024, 2, 18, 0, 0, tzinfo=datetime.timezone.utc), status='new', source='mo_mshp', charges_description='1) FELONY DWI - HABITUAL 2) EXCEEDED POSTED SPEED LIMIT BY 16-19 MPH 3) OPERATE VEH ON HWY WITHOUT VALID LICENSE - 1ST OFFENSE', disposed=False, carrier=None, notes=None, violation=None, court=None, accidentCheckbox=False, commercialDriverLicence=False, ticket_img=None, user_id=None, report=None, details=None, lead_source=None, cloudtalk_upload=False)

In [36]:
lead.state